# Installating R on Jupyter-Portable (www.portabledevapps.net) via a Notebook


In [ ]:
import os
import sys
import io

In [ ]:
try:
    import urllib.request as urllib2  # Python 3
except:
    import urllib2  # Python 2
    
R_exe="R-Portable_3.6.2.paf.exe"
R_url="https://vorboss.dl.sourceforge.net/project/rportable/R-Portable/3.6.2/R-Portable_3.6.2.paf.exe"
hashes="d6f917e9daf1d5a552bf1f9ca8b39a42205d1e78cfcee315bf17963d7e9f2ec0" # sha256
    
R_installer=os.environ["APPSDIR"]+"\\datafiles\\"+R_exe

# We neet 7zip to extract the Installer. If the Julia kernel is installed, it will be in apps\Julia-<Version>\libexec
p7zexe=os.environ["APPSDIR"]+"\\Julia-1.3.1\\libexec\\7z.exe"

In [ ]:
# start download
g = urllib2.urlopen(R_url) 
with io.open(R_installer, 'wb') as f:
    f.write(g.read())
g.close
g = None

In [ ]:
# checking the hashes
import hashlib
def give_hash(of_file, with_this):
    with io.open(R_installer, 'rb') as f:
        return with_this(f.read()).hexdigest()  
print (" "*30+"SHA-256"+" "*30+"\n"+"-"*65)

print ("%s %s" % (give_hash(R_installer, hashlib.sha256),R_installer))
assert give_hash(R_installer, hashlib.sha256) == hashes

In [ ]:
#extract the installer with 7z
from subprocess import run
extr_path = os.environ["APPSDIR"]+"\\R-Portable"
run(p7zexe+' x '+ R_installer + ' -x!$* -o' + extr_path) # do not extract the $PLUGINSDIR folder

In [ ]:
# writing a R script to install necessary packages
Rscript_text = r"""
install.packages(c('stringi'),repos="https://cloud.r-project.org/",type="binary")
install.packages(c('repr', 'IRdisplay', 'evaluate', 'crayon', 'pbdZMQ', 'devtools', 'uuid', 'digest'),repos="https://cloud.r-project.org/",type="binary")
devtools::install_github('IRkernel/IRkernel')
IRkernel::installspec()
"""
if sys.version_info[0] == 3:
    with io.open("temp1.R", 'w', encoding = sys.getdefaultencoding() ) as f:
        for line in Rscript_text.splitlines():
            f.write('%s\n' %  line  )
else:
    with io.open("temp1.R", 'wb'  ) as f:
        for line in Rscript_text.splitlines():
            f.write('%s\r\n' %  line.encode(sys.getdefaultencoding()) )

In [ ]:
# call R script
Rscript_exe = os.environ["APPSDIR"]+"\\R-Portable\\App\\R-Portable\\bin\\Rscript.exe"
#subprocess.call(Rscript_exe+" temp1.R")
run(Rscript_exe+" temp1.R")

In [ ]:
# install kernel.json manually
kernel_path=os.environ["APPSDIR"]+"\\share\\jupyter\\kernels\\R"
os.mkdir(kernel_path)
Rkernel_text = r"""
{
"argv": [
            "{prefix}/R-Portable/App/R-Portable/bin/x64/R.exe",
            "--slave",
            "-e",
            "IRkernel::main()",
            "--args",
            "{connection_file}"],
 "display_name":"R",
 "language": "R"
}
"""

# copy js and png file from default installation
os.rename(os.environ["APPSDIR"]+"\\datafiles\\kernels\\ir\\logo-64x64.png",os.environ["APPSDIR"]+"\\share\\jupyter\\kernels\\R\\logo-64x64.png")
os.rename(os.environ["APPSDIR"]+"\\datafiles\\kernels\\ir\\kernel.js",os.environ["APPSDIR"]+"\\share\\jupyter\\kernels\\R\\kernel.js")
os.remove(os.environ["APPSDIR"]+"\\datafiles\\kernels\\ir\\kernel.json")
os.rmdir(os.environ["APPSDIR"]+"\\datafiles\\kernels\\ir")

if sys.version_info[0] == 3:
    with io.open(kernel_path+"\\kernel.json", 'w', encoding = sys.getdefaultencoding() ) as f:
        for line in Rkernel_text.splitlines():
            f.write('%s\n' %  line  )
else:
    with io.open(kernel_path+"\\kernel.json", 'wb'  ) as f:
        for line in Rkernel_text.splitlines():
            f.write('%s\r\n' %  line.encode(sys.getdefaultencoding()) )

In [ ]:
# delete setup files
os.remove(R_installer)
os.remove("temp1.R")